In [8]:
import numpy as np
import torch
import os
from torch.utils.data import Dataset
from pathlib import Path
from matplotlib import pyplot as plt
from torchvision import transforms

In [7]:
class CANDataset(Dataset):
    def __init__(self, root_dir, is_train=True, transform=None):
        self.root_dir = Path(root_dir) / ('train' if is_train else 'val')
        self.is_train = is_train
        self.transform = transform
        self.total_size = len(os.listdir(self.root_dir))
            
    def __getitem__(self, idx):
        filename = f'{idx}.npz'
        filename = self.root_dir / filename
        data = np.load(filename)
        X, y = data['X'], data['y']
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)
        if self.transform:
            X_tensor = self.transform(X_tensor)
        return X_tensor, y_tensor
    
    def __len__(self):
        return self.total_size

In [16]:
data_dir = '../Data/CHD_w29_s14_ID_Data/1/'
transform = transforms.Normalize(
                            mean=(126.8058,  10.4403,   8.1874,   7.2068,  13.9896,   9.2265,  10.9938, 4.6789, 9.6320), 
                            std =(510.3837,  67.7702,  43.0419,  53.2845,  79.1804,  60.3768,  60.1881, 48.7489,  70.4148))
train_dataset = CANDataset(root_dir=data_dir, is_train=True, transform=transform)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=128,
    shuffle=True, num_workers=8, 
    pin_memory=True, sampler=None
)

In [17]:
X, y = next(iter(train_dataloader))

In [6]:
filename = '../Data/CHD_w29_s14_ID_Data/1/train/1.npz'
data = np.load(filename)
X, y = data['X'], data['y']

In [74]:
train_dataloader.batch_size

128

In [4]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        this_batch_size = data.size()[0]
        weight = this_batch_size / dataloader.batch_size
        # Mean over batch, height and width, but not over the channels
        channels_sum += weight * torch.mean(data, dim=[0,2,3])
        channels_squared_sum += weight * torch.mean(data**2, dim=[0,2,3])
        num_batches += weight
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [5]:
means, stds = get_mean_and_std(train_dataloader)

In [6]:
print('Mean: ', means)
print('Std: ', stds)

Mean:  tensor([126.8058,  10.4403,   8.1874,   7.2068,  13.9896,   9.2265,  10.9938,
          4.6789,   9.6320])
Std:  tensor([510.3837,  67.7702,  43.0419,  53.2845,  79.1804,  60.3768,  60.1881,
         48.7489,  70.4148])
